In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Change directory to your working directory
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/NLP Project"

os.chdir(DATA_DIR)
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/NLP Project'

In [3]:
%pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [52]:
df = pd.read_csv("spoken_written_full_dataset.csv")  # Ensure dataset has 'text' and 'label' columns
df = df.dropna()  # Drop missing values if any
df = df.rename(columns={"type": "label"})  # Renames "type" column to "label"
df

,text,source,formality,label
0,only the portion of the investment that was on...,enron_emails,formal,written
1,"f oh, uh-huh. c because,",telephone_convos,informal,spoken
2,"i congratulate you, mr. president, on your ele...",un_debates,formal,spoken
3,"however, fasb would not allow goodwill amortiz...",enron_emails,formal,written
4,(b) enhanced technical capacity of eclac membe...,un_corpus,formal,written
...,...,...,...,...
2881,core value deadlines self-taught lesson deadli...,textbooks,educational,written
2882,yeah.,telephone_convos,informal,spoken
2883,the united nations human rights council and th...,un_corpus,formal,written
2884,teaming up with arkane ampnet is using the aet...,medium,semiformal,written


In [68]:
import pandas as pd

# Select only the two required columns (e.g., "text" and "label")
selected_columns = ["text", "label"]
df_subset = df[selected_columns]

# Randomly select 500 rows with a fixed seed for reproducibility
df_sampled = df_subset.sample(n=500, random_state=42)

# Reset index for cleanliness
df_sampled = df_sampled.reset_index(drop=True)

# Display the new DataFrame
print(df_sampled.head())

                                                text    label
0  would you please provide me with some examples...  written
1  book title sealed by a kiss author ava gray de...  written
2  convention on the prohibition of the developme...  written
3  motion carries. okay. thank you, madam. item 1...   spoken
4  the voices of indigenous peoples must be heard...  written


In [9]:
id2label = {0: "written", 1: "spoken"}
label2id = {"written": 0, "spoken": 1}

In [91]:
df_sampled["label"] = df_sampled["label"].map(label2id)

In [92]:
dataset = Dataset.from_pandas(df_sampled)
# Split into train/test sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [93]:
train_dataset[0]

{'text': '9. the special rapporteur has continued to place particular emphasis on cooperation with all bodies of the united nations and other regional intergovernmental human rights organizations.',
 'label': 0}

In [94]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [95]:
train_dataset['label'][0]

0

In [96]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [33]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00


In [34]:
import evaluate

accuracy = evaluate.load("accuracy")

In [98]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(train_dataset) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [99]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [100]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    eval_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [101]:
import tensorflow as tf

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [102]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [11]:
!pip install wandb

In [12]:
! wandb login
# API KEY = 8544b034b0ba783804d4ba2d8bcdbeb3fe2de75c

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivanipuli (shivanipuli-university-of-chicago) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [103]:
import wandb
from transformers import TrainingArguments

# ✅ Initialize W&B
wandb.init(project="bert-written-classification")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# ✅ Modify Training Arguments to enable logging
training_args = TrainingArguments(
    output_dir="./hugging_face_bert_classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,  # Log every 10 steps
    report_to="wandb",  # ✅ Enables logging to W&B
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    load_best_model_at_end=True,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [104]:
training_args = TrainingArguments(
    output_dir="./hugging_face_bert_classifier",
    run_name="bert_finetune_spoken_vs_written",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

In [105]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [106]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-106-eb4c7ea1fbf5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.329788,0.870000,0.894493,0.870000,0.865493
2,No log,0.392939,0.890000,0.908060,0.890000,0.887062


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.329788,0.870000,0.894493,0.870000,0.865493
2,No log,0.392939,0.890000,0.908060,0.890000,0.887062


In [108]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.2932603061199188, 'eval_accuracy': 0.93, 'eval_precision': 0.9377777777777777, 'eval_recall': 0.93, 'eval_f1': 0.9290341321713871, 'eval_runtime': 56.7187, 'eval_samples_per_second': 1.763, 'eval_steps_per_second': 0.123, 'epoch': 3.0}


In [117]:
trainer.save_model("./bert_finetuned_spoken_vs_written")  # ✅ Saves the model
tokenizer.save_pretrained("./bert_finetuned_spoken_vs_written")  # ✅ Saves tokenizer

('./bert_finetuned_spoken_vs_written/tokenizer_config.json',
 './bert_finetuned_spoken_vs_written/special_tokens_map.json',
 './bert_finetuned_spoken_vs_written/vocab.txt',
 './bert_finetuned_spoken_vs_written/added_tokens.json',
 './bert_finetuned_spoken_vs_written/tokenizer.json')

In [109]:
trainer.save_state()  # ✅ Saves optimizer, scheduler, and Trainer state

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# ✅ Load Model and Tokenizer
model_path = "/Users/Shivani/nlpfinal/bert_finetuned_spoken_vs_written/"
model2 = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer2 = AutoTokenizer.from_pretrained(model_path)


In [113]:
eval_results = trainer.evaluate()
print(eval_results)

NameError: name 'trainer2' is not defined

In [10]:
import torch

# ✅ Sample input for testing
text = "Uh, so, like, I think this might be, you know, a good idea."
inputs = tokenizer2(text, return_tensors="pt")

# ✅ Predict using the reloaded model
model2.eval()
with torch.no_grad():
    logits = model2(**inputs).logits

predicted_class_id = logits.argmax().item()
print(f"Predicted Label: {id2label[predicted_class_id]}")


Predicted Label: spoken


In [11]:
# ✅ Sample input for testing
text = "The contextual evidence relating to the study (often relating to academics) indicates otherwise."
inputs = tokenizer2(text, return_tensors="pt")

# ✅ Predict using the reloaded model
model2.eval()
with torch.no_grad():
    logits = model2(**inputs).logits

predicted_class_id = logits.argmax().item()
print(f"Predicted Label: {id2label[predicted_class_id]}")

Predicted Label: written
